In [1]:
#Import packages, change directory, inspect data
import pandas as pd
import re
import os
import datetime
os.chdir('C:\\Users\\mhous\\CT\\CT-Drug-Deaths')
df = pd.read_csv('Accidental_Drug_Related_Deaths_2012-2018.csv', index_col = ['ID'])
df.head()

,Date,DateType,Age,Sex,Race,ResidenceCity,ResidenceCounty,ResidenceState,DeathCity,DeathCounty,...,Tramad,Morphine_NotHeroin,Hydromorphone,Other,OpiateNOS,AnyOpioid,MannerofDeath,DeathCityGeo,ResidenceCityGeo,InjuryCityGeo
ID,,,,,,,,,,,,,,,,,,,,,
14-0273,06/28/2014 12:00:00 AM,DateReported,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,Accident,"CT\n(41.575155, -72.738288)","CT\n(41.575155, -72.738288)","CT\n(41.575155, -72.738288)"
13-0102,03/21/2013 12:00:00 AM,DateofDeath,48.0,Male,Black,NORWALK,NaN,NaN,NORWALK,FAIRFIELD,...,NaN,NaN,NaN,NaN,NaN,NaN,Accident,"Norwalk, CT\n(41.11805, -73.412906)","NORWALK, CT\n(41.11805, -73.412906)","CT\n(41.575155, -72.738288)"
16-0165,03/13/2016 12:00:00 AM,DateofDeath,30.0,Female,White,SANDY HOOK,FAIRFIELD,CT,DANBURY,NaN,...,NaN,NaN,NaN,NaN,NaN,Y,Accident,"Danbury, CT\n(41.393666, -73.451539)","SANDY HOOK, CT\n(41.419998, -73.282501)",NaN
16-0208,03/31/2016 12:00:00 AM,DateofDeath,23.0,Male,White,RYE,WESTCHESTER,NY,GREENWICH,NaN,...,NaN,NaN,NaN,NaN,NaN,Y,Accident,"Greenwich, CT\n(41.026526, -73.628549)",NaN,NaN
13-0052,02/13/2013 12:00:00 AM,DateofDeath,22.0,Male,"Asian, Other",FLUSHING,QUEENS,NaN,GREENWICH,FAIRFIELD,...,NaN,NaN,NaN,NaN,NaN,NaN,Accident,"Greenwich, CT\n(41.026526, -73.628549)",NaN,"CT\n(41.575155, -72.738288)"


Data comes from https://data.ct.gov/Health-and-Human-Services/Accidental-Drug-Related-Deaths-2012-2018/rybz-nyjw

The purpose of this mini-project is to practice the principles data cleaning, exploratory data analysis and data visualization

In [2]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 5105 entries, 14-0273 to 16-0637
Data columns (total 40 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Date                 5103 non-null   object 
 1   DateType             5103 non-null   object 
 2   Age                  5102 non-null   float64
 3   Sex                  5099 non-null   object 
 4   Race                 5092 non-null   object 
 5   ResidenceCity        4932 non-null   object 
 6   ResidenceCounty      4308 non-null   object 
 7   ResidenceState       3556 non-null   object 
 8   DeathCity            5100 non-null   object 
 9   DeathCounty          4005 non-null   object 
 10  Location             5081 non-null   object 
 11  LocationifOther      590 non-null    object 
 12  DescriptionofInjury  4325 non-null   object 
 13  InjuryPlace          5039 non-null   object 
 14  InjuryCity           3349 non-null   object 
 15  InjuryCounty         2364 non-null

We have 40 columns. The first 19 are descriptions of the victim and their whereabouts, the next 15 are indicator variables that tell us if that drug was found in their system, and the last 4 provide more detail about the geographic coordinates and manner of death.

Let's start by inspecting the Date column

In [3]:
df['Date'].head()

ID
14-0273    06/28/2014 12:00:00 AM
13-0102    03/21/2013 12:00:00 AM
16-0165    03/13/2016 12:00:00 AM
16-0208    03/31/2016 12:00:00 AM
13-0052    02/13/2013 12:00:00 AM
Name: Date, dtype: object

Let's make sure that every time of death is midnight 

In [4]:
df['Date'].str.split().str[-2].value_counts()

12:00:00    5103
Name: Date, dtype: int64

Since every time of death is midnight, so we ignore that when converting the date column to type datetime. We also create a year column for visualization purposes.

In [5]:
df['Date'] = pd.to_datetime(df['Date'])
df['Year'] = df['Date'].dt.year

Let's look at some of the descriptive columns. We note that ResidenceCity, ResidenceCounty, DeathCity, and DeathCounty are all in uppercase, which is a little unsightly. We correct this by setting each column to title case.

In [6]:
uppercase_columns = ['ResidenceCity', 'ResidenceCounty', 'DeathCity', 'DeathCounty']

df[uppercase_columns].head()

,ResidenceCity,ResidenceCounty,DeathCity,DeathCounty
ID,,,,
14-0273,NaN,NaN,NaN,NaN
13-0102,NORWALK,NaN,NORWALK,FAIRFIELD
16-0165,SANDY HOOK,FAIRFIELD,DANBURY,NaN
16-0208,RYE,WESTCHESTER,GREENWICH,NaN
13-0052,FLUSHING,QUEENS,GREENWICH,FAIRFIELD


In [7]:
for i in uppercase_columns:
    df[i] = df[i].str.title()
    
df[uppercase_columns].head()

,ResidenceCity,ResidenceCounty,DeathCity,DeathCounty
ID,,,,
14-0273,NaN,NaN,NaN,NaN
13-0102,Norwalk,NaN,Norwalk,Fairfield
16-0165,Sandy Hook,Fairfield,Danbury,NaN
16-0208,Rye,Westchester,Greenwich,NaN
13-0052,Flushing,Queens,Greenwich,Fairfield


In [8]:
df['DeathCounty'].value_counts()

Hartford      1233
New Haven     1107
Fairfield      623
New London     368
Litchfield     237
Middlesex      181
Windham        142
Tolland        113
Usa              1
Name: DeathCounty, dtype: int64

Since Usa is not a county, let's look at the observation and see if we can fix it.

In [9]:
df[df['DeathCounty'] == 'Usa'] 

,Date,DateType,Age,Sex,Race,ResidenceCity,ResidenceCounty,ResidenceState,DeathCity,DeathCounty,...,Morphine_NotHeroin,Hydromorphone,Other,OpiateNOS,AnyOpioid,MannerofDeath,DeathCityGeo,ResidenceCityGeo,InjuryCityGeo,Year
ID,,,,,,,,,,,,,,,,,,,,,
13-0339,2013-10-07,DateofDeath,56.0,Female,White,Hartford,Usa,NaN,Hartford,Usa,...,NaN,NaN,NaN,NaN,NaN,Accident,"Hartford, CT\n(41.765775, -72.673356)","HARTFORD, CT\n(41.765775, -72.673356)","CT\n(41.575155, -72.738288)",2013.0


Since both the ResidencyCity and DeathCity is Hartford, that means the ResidenceCounty and DeathCounty are Hartford County. Let's replace  'Usa' with 'Hartford'.


In [10]:
df['DeathCounty'] = df['DeathCounty'].replace('Usa', 'Hartford')
df['ResidenceCounty'] = df['ResidenceCounty'].replace('Usa', 'Hartford')
df['DeathCounty'].value_counts()

Hartford      1234
New Haven     1107
Fairfield      623
New London     368
Litchfield     237
Middlesex      181
Windham        142
Tolland        113
Name: DeathCounty, dtype: int64

The DateType column is consistent, so no cleaning necessary

In [11]:
df['DateType'].value_counts()

DateofDeath     2822
DateReported    2281
Name: DateType, dtype: int64

In [12]:
df['MannerofDeath'].value_counts()

Accident    5066
Pending       14
accident      13
Natural        1
ACCIDENT       1
Name: MannerofDeath, dtype: int64

Since there are multiple spellings of 'Accident', let's merge them.

In [13]:
df['MannerofDeath'] = df['MannerofDeath'].replace('ACCIDENT', 'Accident')
df['MannerofDeath'] = df['MannerofDeath'].replace('accident', 'Accident')
df['MannerofDeath'].value_counts()

Accident    5080
Pending       14
Natural        1
Name: MannerofDeath, dtype: int64

In [14]:
#Inspect what the Natural death is
print(df[df['MannerofDeath']=='Natural']['COD'], df[df['MannerofDeath']=='Natural']['OtherSignifican'])

ID
12-0276    Oxycodone Toxicity
Name: COD, dtype: object ID
12-0276    Atherosclerotic Cardiovascular Disease
Name: OtherSignifican, dtype: object


The dataset is specifically described as 'Accidental Drug Related Deaths 2012-2018', so if the autopsy says the cause of death was natural, then there's no need to change anything.

Let's check some more columns to see if they're clean or not

In [15]:
df['Location'].value_counts()

Residence            2677
Hospital             1626
Other                 773
Convalescent Home       3
Nursing Home            1
Hospice                 1
Name: Location, dtype: int64

In [16]:
df['Race'].value_counts()

White                     4004
Hispanic, White            561
Black                      433
Hispanic, Black             24
Unknown                     23
Asian, Other                18
Asian Indian                14
Other                       11
Chinese                      2
Hawaiian                     1
Native American, Other       1
Name: Race, dtype: int64

In [17]:
df['Sex'].value_counts()

Male       3773
Female     1325
Unknown       1
Name: Sex, dtype: int64

Location, Race and Sex are all consistent, so no cleaning can be done.

In [18]:
df['DescriptionofInjury'].value_counts().head(10)

Substance Abuse    1618
Drug Use            491
Substance abuse     402
Ingestion           334
substance abuse     298
Drug abuse          203
Injection           158
Inhalation          113
SUBSTANCE ABUSE      66
drug use             62
Name: DescriptionofInjury, dtype: int64

This variable probably works best when describing an individual's death, and not for a broad understanding of the data. I also don't have domain specific knowledge, so I'm not sure if there is really a substantive difference between Substance Abuse and Drug Use. 

Let's now convert the indicator variables the have 1/0s. 

In [19]:
#Since other is different from the others, we have to clean that separately
df['Other'] = df['Other'].replace('.', 1,regex=True).fillna(0).astype(int, errors = 'ignore')
drug_columns = list(df.columns[19:36])

#Fill any NaNs with a 0, and any words with a 1
df[drug_columns] = df[drug_columns].fillna(0).replace(r'\w', 1, regex = True)
df[drug_columns].head(2)

,Heroin,Cocaine,Fentanyl,FentanylAnalogue,Oxycodone,Oxymorphone,Ethanol,Hydrocodone,Benzodiazepine,Methadone,Amphet,Tramad,Morphine_NotHeroin,Hydromorphone,Other,OpiateNOS,AnyOpioid
ID,,,,,,,,,,,,,,,,,
14-0273,0,0,1,0,0,0,0,1,1,0,0,0,0,0,0,0,0
13-0102,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


Let's also create an indicator variable for if the Description of Injury contains words starting with [Mm]ed. We choose [Mm]ed because when we inspected the data we found that some descriptions said "Meds" or "meds", and also "Medicine" or "Medication"

In [20]:
df['Medication'] = '0'
mask_med = df['DescriptionofInjury'].str.contains('[Mm]ed+') == True
df.loc[mask_med, 'Medication'] = 1
df['Medication'].value_counts()

0    4894
1     211
Name: Medication, dtype: int64

In [21]:
#Make sure the drug columns are integers
df[drug_columns] = df[drug_columns].astype(int, errors='ignore')

We can also create a column that indicates the number of drugs that were found in the victim's body

In [22]:
df['Number_of_drugs'] = df[drug_columns].sum(axis=1)
print(df['Number_of_drugs'].value_counts())

2    1490
3    1392
1     882
4     873
5     345
6      79
7      24
0      18
8       2
Name: Number_of_drugs, dtype: int64


It doesn't make sense for there to be accidental drug deaths where there were no drugs found in the system, so let's look at the cause of death for the observations that appear to have 0 drugs in their system. 

In [23]:
df[df['Number_of_drugs'] == 0]['COD']

ID
15-0314                                  ethanol, clonazepam
18-0641                          Acute Ketamine Intoxication
18-0610                            Acute Opioid Intoxication
13-0236                   Massive Pontine Hemorrhage/cocaine
14-0523                                  Opiate Intoxication
18-0935                          Acute Ketamine Intoxication
18-0855                Acute 1,1-Difluoroethane Intoxication
13-0249    Intoxication due to the Combined Effects of Mo...
18-0752                      Acute Cocaethylene Intoxication
15-0296    Cocaine induced Ruptured Cerebral Aneurysm COC...
17-0137    Combined Effects of Lamotrigine, Quetiapine, S...
12-0055                               Multiple Drug Toxicity
15-0024    Intoxication due to the combined effects of Et...
17-0906    Hypertensive Cardiovascular Disease  and Acute...
13-0305    Acute Intoxiation due to the combined effects ...
13-0023          Acute and Chronic Substance Abuse (Opiates)
18-0569              

Since there's only 18 observations like this, is probably isn't necessary to manually fix each observation, but we still decided to do so. 

In [24]:
df.loc['15-0314', 'Benzodiazepine'] = 1
df.loc['15-0314', 'Ethanol'] = 1
df.loc['18-0641', 'Other'] = 1
df.loc['18-0610', 'AnyOpioid'] = 1
df.loc['13-0236', 'Cocaine'] = 1
df.loc['14-0523', 'AnyOpioid'] = 1
df.loc['18-0935', 'Other'] = 1
df.loc['18-0855', 'Other'] = 1
df.loc['13-0249', 'Morphine_NotHeroin']  = 1
df.loc['13-0249', 'Other']  = 1
df.loc['18-0752', 'Other'] = 1
df.loc['15-0296', 'Cocaine'] = 1
df.loc['17-0137', 'Other'] = 1
df.loc['12-0055', 'Other'] = 1
df.loc['15-0024', 'Ethanol'] = 1
df.loc['15-0024', 'Other'] = 1
df.loc['17-0906', 'Other'] = 1
df.loc['13-0305', 'AnyOpioid'] = 1
df.loc['13-0305', 'Benzodiazepine'] = 1
df.loc['13-0023', 'AnyOpioid'] = 1
df.loc['18-0569', 'AnyOpioid'] = 1
df.loc['17-0894', 'Other'] = 1

In [25]:
df['Number_of_drugs'] = df[drug_columns].sum(axis=1)
print(df['Number_of_drugs'].value_counts())

2    1494
3    1392
1     896
4     873
5     345
6      79
7      24
8       2
Name: Number_of_drugs, dtype: int64


In [26]:
df.columns

Index(['Date', 'DateType', 'Age', 'Sex', 'Race', 'ResidenceCity',
       'ResidenceCounty', 'ResidenceState', 'DeathCity', 'DeathCounty',
       'Location', 'LocationifOther', 'DescriptionofInjury', 'InjuryPlace',
       'InjuryCity', 'InjuryCounty', 'InjuryState', 'COD', 'OtherSignifican',
       'Heroin', 'Cocaine', 'Fentanyl', 'FentanylAnalogue', 'Oxycodone',
       'Oxymorphone', 'Ethanol', 'Hydrocodone', 'Benzodiazepine', 'Methadone',
       'Amphet', 'Tramad', 'Morphine_NotHeroin', 'Hydromorphone', 'Other',
       'OpiateNOS', 'AnyOpioid', 'MannerofDeath', 'DeathCityGeo',
       'ResidenceCityGeo', 'InjuryCityGeo', 'Year', 'Medication',
       'Number_of_drugs'],
      dtype='object')

All done with cleaning, let's just reindex the columns so they're in more logical locations

In [27]:
new_columns = ['Date', 'Year', 'DateType', 'Age', 'Sex', 'Race', 'ResidenceCity',
       'ResidenceCounty', 'ResidenceState', 'DeathCity', 'DeathCounty',
       'Location', 'LocationifOther', 'DescriptionofInjury', 'InjuryPlace',
       'InjuryCity', 'InjuryCounty', 'InjuryState', 'COD', 'OtherSignifican',
       'Heroin', 'Cocaine', 'Fentanyl', 'FentanylAnalogue', 'Oxycodone',
       'Oxymorphone', 'Ethanol', 'Hydrocodone', 'Benzodiazepine', 'Methadone',
       'Amphet', 'Tramad', 'Morphine_NotHeroin', 'Hydromorphone', 'Other',
       'OpiateNOS', 'AnyOpioid', 'Medication', 'Number_of_drugs', 'MannerofDeath', 'DeathCityGeo',
       'ResidenceCityGeo', 'InjuryCityGeo']
df = df.reindex(columns=new_columns)

And finally let's export it to csv so we can use it for data visualization in another notebook

In [28]:
df.to_csv(r'C:\\Users\\mhous\\CT\\CT-Drug-Deaths\\drug_deaths_clean.csv')